In [1]:
import ipywidgets
import traitlets
import yt

In [2]:
ds = yt.load("data/IsolatedGalaxy/galaxy0030/galaxy0030")

yt : [INFO     ] 2018-03-16 20:39:29,507 Parameters: current_time              = 0.0060000200028297998
yt : [INFO     ] 2018-03-16 20:39:29,508 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2018-03-16 20:39:29,509 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2018-03-16 20:39:29,510 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2018-03-16 20:39:29,511 Parameters: cosmological_simulation   = 0.0


In [88]:
def display_arr(arr):
    unit_registry = arr.units.registry
    dimension = arr.units.dimensions
    equiv = [k for k, v in unit_registry.lut.items() if v[1] is dimension]
    dropdown = ipywidgets.Dropdown(options = sorted(equiv), value = str(arr.units))
    def arr_updater(arr, texts):
        def _value_updater(change):
            arr2 = arr.in_units(change['new'])
            if arr2.shape == ():
                arr2 = [arr2]
            for v, t in zip(arr2, texts):
                t.value = str(v.value)
        return _value_updater
    if arr.shape == ():
        arr_iter = [arr]
    else:
        arr_iter = arr
    texts = [ipywidgets.Text(value = str(_.value), disabled = True) for _ in arr_iter]
    dropdown.observe(arr_updater(arr, texts), names="value")
    return ipywidgets.HBox(texts + [dropdown])

In [89]:
display_arr(ds.domain_width)

HBox(children=(Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length')))

In [94]:
def display_ds(ds):
    attrs = ["domain_width", "domain_left_edge", "domain_right_edge", "current_time"]
    lb = ipywidgets.VBox([ ipywidgets.Label(value = _) for _ in attrs])
    vb = ipywidgets.VBox([ display_arr(getattr(ds, _)) for _ in attrs])
    hb = ipywidgets.HBox([lb, vb])
    return hb

In [95]:
display_ds(ds)

HBox(children=(VBox(children=(Label(value='domain_width'), Label(value='domain_left_edge'), Label(value='domain_right_edge'), Label(value='current_time'))), VBox(children=(HBox(children=(Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length'))), HBox(children=(Text(value='0.0', disabled=True), Text(value='0.0', disabled=True), Text(value='0.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length'))), HBox(children=(Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length'))), HBox(children=(Text(value='0.0060000200028298', disabled=True), Dropdown(options=('code_time', 'd', 'day', 'hr', 'min', 's', 't_geom', 't_pl', 'yr'), value='code_time')))))))

In [96]:
import ipywidgets as widgets # Loads the Widget framework.
from IPython.core.magics.namespace import NamespaceMagics # Used to query namespace.

# For this example, hide these names, just to avoid polluting the namespace further
get_ipython().user_ns_hidden['widgets'] = widgets
get_ipython().user_ns_hidden['NamespaceMagics'] = NamespaceMagics

In [97]:
class VariableInspectorWindow(object):
    instance = None

    def __init__(self, ipython):
        """Public constructor."""
        if VariableInspectorWindow.instance is not None:
            raise Exception("""Only one instance of the Variable Inspector can exist at a
                time.  Call close() on the active instance before creating a new instance.
                If you have lost the handle to the active instance, you can re-obtain it
                via `VariableInspectorWindow.instance`.""")

        VariableInspectorWindow.instance = self
        self.closed = False
        self.namespace = NamespaceMagics()
        self.namespace.shell = ipython.kernel.shell

        self._box = widgets.Box()
        self._box.layout.overflow_y = 'scroll'
        self._table = widgets.HTML(value = 'Not hooked')
        self._box.children = [self._table]

        self._ipython = ipython
        self._ipython.events.register('post_run_cell', self._fill)

    def close(self):
        """Close and remove hooks."""
        if not self.closed:
            self._ipython.events.unregister('post_run_cell', self._fill)
            self._box.close()
            self.closed = True
            VariableInspectorWindow.instance = None

    def _fill(self):
        """Fill self with variable information."""
        values = self.namespace.who_ls()
        self._table.value = '<div class="rendered_html jp-RenderedHTMLCommon"><table><thead><tr><th>Name</th><th>Type</th><th>Value</th></tr></thead><tr><td>' + \
            '</td></tr><tr><td>'.join(['{0}</td><td>{1}</td><td>{2}'.format(v, type(eval(v)).__name__, str(eval(v))) for v in values]) + \
            '</td></tr></table></div>'

    def _ipython_display_(self):
        """Called when display() or pyout is used to display the Variable
        Inspector."""
        self._box._ipython_display_()


In [98]:
inspector = VariableInspectorWindow(get_ipython())
inspector

Box(children=(HTML(value='Not hooked'),), layout=Layout(overflow_y='scroll'))

In [99]:
v = 'this'